In [1]:
#Import Libraries
import random as rn
import yfinance as yf  
import numpy as np
import pandas as pd
import py_vollib.black.greeks.analytical as bs #Special Package to calculate deltas - MIT License
import pandasql
from numpy import sqrt,mean,log,diff
from longstaff_schwartz.algorithm import longstaff_schwartz
from longstaff_schwartz.stochastic_process import GeometricBrownianMotion
from longstaff_schwartz.binomial import create_binomial_model, american_put_price, american_put_exercise_barrier_fitted

In [ ]:
################### START HERE ######################

In [2]:
class data_download:
    
    def __init__(self, symbol, start_dt, end_dt, option_calendar_days, splits):
        
        self.symbol = symbol
        self.start_dt = start_dt
        self.end_dt = end_dt
        self.data = None
        self.opt_cal_days = option_calendar_days
        self.splits = splits
        self.data_dict = {}
        
    def download(self):
        self.data = yf.download(self.symbol,self.start_dt,self.end_dt)
        
    def generate_splits(self):
        for h in range(1, self.splits):
            self.data_dict[h] = self.data.iloc[self.opt_cal_days*(h-1) : self.opt_cal_days*(h-1)+(self.opt_cal_days-1), 4].values.flatten()


In [3]:
#voo = data_download('VOO','2010-01-01','2019-08-01',62,35)

voo = data_download('VOO','2017-04-17','2020-03-31',62,12)
voo.download()
voo.generate_splits()

[*********************100%***********************]  1 of 1 completed


In [4]:
voo.data_dict[12] = np.array([293.661407,
294.417847,
297.025635,
294.855774,
295.960602,
295.14444,
296.667297,
298.717651,
297.861694,
299.832428,
299.434296,
300.101166,
302.609406,
303.634583,
303.007507,
303.087158,
303.385742,
300.618744,
295.841187,
299.046143,
298.747528,
299.683136,
294.30838,
296.448303,
301.016876,
304.420898,
305.456055,
303.90332,
306.083069,
306.640442,
308.611206,
308.312622,
308.8302,
307.984161,
309.467224,
308.252899,
305.057892,
294.855774,
285.997375,
284.733307,
272.112579,
270.470276,
282.543579,
274.750183,
286.136749,
276.551727,
271.913513,
250.931992,
263.809998,
250.729996,
226.479996,
247.639999,
218.570007,
232.880005,
220.149994,
220.339996,
210.740005,
204.270004,
223.75,
226.619995,
240.240005])

In [5]:
class data_prep:
    
    def __init__ (self, prev_prices, prices, strike_price, state_cut_offs = [0.5, 1.0], time_cut_offs = [3,7],option_length_years = 0.25, risk_free = 0.05, symbol = 'VOO'):
        
        self.symbol = symbol
        self.prices = prices
        self.prev_prices = prev_prices
        self.est_daily_volatility = None
        self.annual_vol = None
        self.strike_price = strike_price
        self.state_cut_offs = state_cut_offs
        self.time_cut_offs = time_cut_offs
        self.option_length = option_length_years
        self.risk_free = risk_free
        self.state_defs = None
    
    def estimate_volatility(self):
        r = diff(log(self.prev_prices))
        r_mean = mean(r)
        diff_square = [(r[i]-r_mean)**2 for i in range(0,len(r))]
        std = sqrt(sum(diff_square)*(1.0/(len(r)-1)))
        self.est_daily_volatility = std #Calculated at Daily Level
        self.annual_vol = self.est_daily_volatility*np.sqrt(252)
        
    def boundary_vector(self):
        mdl = create_binomial_model(sigma=self.est_daily_volatility, r=0.0005, S0=self.prices[0], T=len(self.prices), n=100)
        exercise_barrier = american_put_exercise_barrier_fitted(mdl, self.strike_price, 3)
        boundary_vec = [exercise_barrier(k) for k in range(1,len(self.prices)+1)]
        return(boundary_vec)
    
    def longstaff_schwartz(self, boundary_policy):
        ls_policy = [ 1 if self.prices[i] > boundary_policy[i] else 0 for i in range(len(self.prices))]
        return(ls_policy)

    

In [ ]:
############## Generate Policy ######################

In [ ]:
########### Prediction ################

In [6]:
class ls_online_predict:
    
    def __init__ (self, data_object, policy):
        
        self.data_object = data_object
        self.policy = policy
        self.epsilon_episode = None
        self.deltas = None
        
    def delta(self):
        self.deltas =  [np.around(bs.delta('c',self.data_object.prices[h], 
                          self.data_object.strike_price, 
                          self.data_object.option_length - np.around(h/252,2), #Time to expiry
                          self.data_object.risk_free, 
                          self.data_object.annual_vol),2) for h in range(len(self.data_object.prices))]
        
    #Here episode represents whether strategy = 0 or 1
    def predict_profit_function(self, episode):
    
        positions = [self.deltas[h] if episode[h] == 1 else self.deltas[h-1] for h in range(len(self.data_object.prices))]
    
        #Cost of Borrowing Money for 1 day 
        dollar_positions = -1*[self.data_object.prices[h]*positions[h]*self.data_object.risk_free for h in range(len(self.data_object.prices))]
    
        #Cost of adjusting portfolio to bring position at par with BS delta
        portfolio_positions = [self.data_object.prices[h]*(positions[h] - positions[h-1]) for h in range(1,len(self.data_object.prices))]
    
        #Sum daily transaction costs over the Option Time Frame
        transaction_cost = sum(dollar_positions)
    
        #Sum daily portfolio costs over the Option Time Frame
        portfolio_cost = sum(portfolio_positions)
    
        #Return the Excess Profit over Black Scholes
        return transaction_cost + portfolio_cost
    
    def ls_predict(self):
        
        self.epsilon_episode = self.policy
        
        return(self.predict_profit_function(self.epsilon_episode))
    
    

In [18]:
ls4 = data_prep(prev_prices=bnd.data_dict[3], prices=bnd.data_dict[4],strike_price=76.000000)

In [19]:
ls4.estimate_volatility()

In [20]:
boundary_v = ls4.boundary_vector()

In [21]:
boundary_pol = ls4.longstaff_schwartz(boundary_v)

In [ ]:
#Predict

In [22]:
L4 = ls_online_predict(ls4,boundary_pol)


In [23]:
L4.delta()


In [24]:
print(L4.ls_predict())


0.2503160858154333


In [7]:
ls4 = data_prep(prev_prices=voo.data_dict[3], prices=voo.data_dict[4],strike_price=250.000000)
ls4.estimate_volatility()
boundary_v = ls4.boundary_vector()
boundary_pol = ls4.longstaff_schwartz(boundary_v)


#Predict
L4 = ls_online_predict(ls4,boundary_pol)
L4.delta()
print(L4.ls_predict())


5.598106231689447


In [ ]:
#####################

In [8]:
ls5 = data_prep(prev_prices=voo.data_dict[11], prices=voo.data_dict[12],strike_price=300.000000)
ls5.estimate_volatility()
boundary_v2 = ls5.boundary_vector()
boundary_pol2 = ls5.longstaff_schwartz(boundary_v2)


#Predict
L5 = ls_online_predict(ls5,boundary_pol2)
L5.delta()
print(L5.ls_predict())


-80.03715406000003


In [ ]:
#####################

In [9]:
ls6 = data_prep(prev_prices=voo.data_dict[5], prices=voo.data_dict[6],strike_price=255.000000)
ls6.estimate_volatility()
boundary_v3 = ls6.boundary_vector()
boundary_pol3 = ls6.longstaff_schwartz(boundary_v3)


#Predict
L6 = ls_online_predict(ls6,boundary_pol3)
L6.delta()
print(L6.ls_predict())

180.64434951782223
